<a href="https://colab.research.google.com/github/Impervious25/UNI/blob/main/ELC_OCR(SEM5_2021).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Experiential Learning activity** for "**Handwritten: Optical Character Recognition**"

Assignment:-
To draw Confusion Matrix (for 10 classes: A - J English Letters) and  Cross Validation (fold 2,3,4,5,6,7,8,9,10) from given raw data of handwritten 10 classes (A - J English Letters) and source code of SVM model designing.

TEAM ->

NAME     :    ROLL
*   **Harneet Kaur**     :  101917189
*   **Aryan Bhardwaj**   :  101917195
*   **Viplav Shrungare** :  101917197
*   **Divyanshu Jain**   :  102097010


In [1]:
import io
import os

import cv2
import glob
import h5py

import keras
import imageio
from PIL import Image

import numpy as np
from numpy import mean, std

from scipy import fftpack, ndimage

import sklearn
from sklearn import svm, metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold, cross_val_predict

import matplotlib.pyplot as plt
import matplotlib.image as mpimage

import datetime as dt
from six.moves import range

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import zipfile

zip_ref=zipfile.ZipFile('/content/drive/MyDrive/ELC-OCR/data/train_EC.zip','r')
zip_ref.extractall('/content/drive/MyDrive/ELC-OCR/extracted_data')

In [4]:
!ls /content/drive/MyDrive/ELC-OCR/extracted_data/train_EC

train_path="/content/drive/MyDrive/ELC-OCR/extracted_data/train_EC"

a  b  c  d  e  f  g  h	i  j


In [5]:
pixel_depth=255.0

train_labels=os.listdir(train_path)
#test_labels=os.listdir(test_path)

train_labels.sort()
#test_labels.sort()

print(train_labels)
nb_classes= 10

global_features_train=[]

train_classes=[]
#test_classes=[]

i,j=0,0
k=0

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']


In [6]:
from matplotlib.pyplot import imread

for training_name in train_labels:
    path=os.path.join(train_path,training_name,'*')
    files=glob.glob(path)
    current_label=training_name
    k=1

    for fl in files:
        image=imread(fl)
        global_feature=np.hstack([image])
        train_classes.append(current_label)
        global_features_train.append(global_feature)
        i+=1
        k+1

    print("[status] processed folder: {}".format(current_label))
    j+=1

print("\n[status] completed global feature extraction...")
print("[status] feature vector size:\t{}".format(np.array(global_features_train).shape))
print("[status] training labels:\t{}".format(np.array(train_classes).shape))

[status] processed folder: a
[status] processed folder: b
[status] processed folder: c
[status] processed folder: d
[status] processed folder: e
[status] processed folder: f
[status] processed folder: g
[status] processed folder: h
[status] processed folder: i
[status] processed folder: j

[status] completed global feature extraction...
[status] feature vector size:	(835, 32, 32)
[status] training labels:	(835,)


In [7]:
#labels = (np.arange(nb_classes) == labels[:,None]).astype(np.float32)

targetNames=np.unique(train_classes)
le=LabelEncoder()
target=le.fit_transform(train_classes)

print("[status] training labels encoded...")

[status] training labels encoded...


In [8]:
#print(np.array(global_features_train).shape)

n_samples, nx, ny=np.array(global_features_train).shape
d2_global_features=np.array(global_features_train).reshape((n_samples, nx*ny))

#scaler=MinMaxScaler(feature_range=(0,1))
#rescaled_features=scaler.fit_transform(d2_global_features)

In [9]:
print("[Status] feature vector normalized...\n")

print("[Status] target label:\n{}".format(target))
print("\n[Status] target label shape: {}".format(target.shape))

[Status] feature vector normalized...

[Status] target label:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4

In [10]:
h5f_data=h5py.File('/content/drive/MyDrive/ELC-OCR/extracted_data/data.h5','w')
h5f_data.create_dataset('dataset_1',data=np.array(d2_global_features))

h5f_label=h5py.File('/content/drive/MyDrive/ELC-OCR/extracted_data/labels.h5','w')
h5f_label.create_dataset('dataset_1',data=np.array(target))

<HDF5 dataset "dataset_1": shape (835,), type "<i8">

In [11]:
h5f_data.close()
h5f_label.close()

In [12]:
h5f_data=h5py.File('/content/drive/MyDrive/ELC-OCR/extracted_data/data.h5','r')
h5f_label=h5py.File('/content/drive/MyDrive/ELC-OCR/extracted_data/labels.h5','r')

In [13]:
global_features_string_train= h5f_data['dataset_1']
global_labels_string_train=h5f_label['dataset_1']

global_features_train=np.array(global_features_string_train)
global_labels_train=np.array(global_labels_string_train)

In [14]:
h5f_data.close()
h5f_label.close()

In [15]:
print("[Status] training feature shape: {}".format(global_features_train.shape))

print("[Status] labels shape: {}".format(global_labels_train.shape))

[Status] training feature shape: (835, 1024)
[Status] labels shape: (835,)


In [16]:
clf = svm.SVC(kernel='linear')

In [17]:
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_predict, KFold

for i in range(2,11):

  print("\n\nFOLD SPLIT: ", i)

  cv = KFold(n_splits=i, random_state=1, shuffle=True)
  scores = cross_val_score(clf,global_features_train,global_labels_train , scoring='accuracy', cv=cv, n_jobs=-1)
  print("\nScores:\n%s" % scores)
  print('\nAccuracy: %.3f (%.3f)\n' % (mean(scores), std(scores)))
  
  #confusion matrix
  y_pred = cross_val_predict(clf, global_features_train,global_labels_train , cv=i)
  conf_mat = confusion_matrix(global_labels_train, y_pred)
  print("Confusion matrix:\n\n%s" % conf_mat)
  print("\n========================================")



FOLD SPLIT:  2

Scores:
[0.79425837 0.76498801]

Accuracy: 0.780 (0.015)

Confusion matrix:

[[82  4  2  5  0  1  1  5  0  0]
 [12 73  1 10  0  3  1  0  0  0]
 [ 0  1 79  2 13  2  1  2  0  0]
 [ 9  9  3 75  0  0  0  1  2  0]
 [ 2  1 11  3 81  1  1  0  0  0]
 [ 1  2  2  1  1 41  1  0  2  3]
 [ 3  5  2  2  1  2 79  2  2  2]
 [21 11  0  3  7  0  1 57  0  0]
 [ 1  0  2  2  0  3  2  0 12  6]
 [ 1  2  2  2  2  6  2  0  4 33]]



FOLD SPLIT:  3

Scores:
[0.83512545 0.79136691 0.78417266]

Accuracy: 0.804 (0.023)

Confusion matrix:

[[88  2  1  5  1  2  0  1  0  0]
 [ 9 78  0  7  1  1  2  1  1  0]
 [ 0  1 90  1  5  1  1  0  0  1]
 [ 7  9  4 74  1  1  0  0  3  0]
 [ 1  0 26  2 67  1  0  3  0  0]
 [ 1  2  1  0  1 43  1  0  2  3]
 [ 1  4  2  2  1  2 85  1  2  0]
 [24 10  1  2  2  0  3 58  0  0]
 [ 1  0  2  2  0  3  2  0 11  7]
 [ 2  1  3  2  1  1  1  0  2 41]]



FOLD SPLIT:  4

Scores:
[0.79904306 0.81818182 0.77990431 0.78846154]

Accuracy: 0.796 (0.014)

Confusion matrix:

[[82  4  1  5  1  